In [22]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
%matplotlib inline
import gc
import sys
sys.path.append('../../GLS/python/')

import lightkurve as lk
from astropy.timeseries import LombScargle
import astropy.units as u

from gls import Gls
from tqdm.notebook import tqdm

from logging import getLogger
logger = getLogger(__name__)


In [85]:
target_list = [
    "TOI540",
    "TOI837",
    "TOI942",
    "TOI251",
    "TOI451",
    "TOI1685",
    "TOI776"
]

P_rot_list = [
    "0.72610",
    "3.004",
    "3.39",
    "3.84",
    "5.1",
    "18.66",
    "34.4"
]

P_rot_var_list = [
    "0.00039",
    "0.053",
    "0.01",
    "0.48",
    "0.1",
    "0.71",
    "2.0"
]
df_preds = pd.DataFrame()
df_preds["target"] = pd.Series(target_list)
df_preds["P_rot"] = pd.Series(P_rot_list)
df_preds["P_rot_var"] = pd.Series(P_rot_var_list)
df_preds["P_rot_pred"] = None
df_preds["P_rot_var_pred"] = None

In [86]:
df_preds

,target,P_rot,P_rot_var,P_rot_pred,P_rot_var_pred
0,TOI540,0.72610,0.00039,None,None
1,TOI837,3.004,0.053,None,None
2,TOI942,3.39,0.01,None,None
3,TOI251,3.84,0.48,None,None
4,TOI451,5.1,0.1,None,None
5,TOI1685,18.66,0.71,None,None
6,TOI776,34.4,2.0,None,None


In [88]:
if __name__ == "__main__":
    P_rot_pred = []
    P_rot_var_pred = []
    for idx, target_star in enumerate(tqdm(target_list)):
        try:
            lc_file = lk.search_lightcurve(f"{target_star}")
            lc = lc_file[lc_file.author.data=="SPOC"][0].download()
            lc = lc.normalize().remove_nans()
            
            gc.collect()

            #calculate the P_rot by GLM
            df = [lc.time.value, lc.flux.value, lc.flux_err.value]
            gls = Gls(df, Pbeg=0.1, Pend=10)
            preds = gls.best
            print(preds)

            #store the predicted values
            P_rot_pred.append(preds["P"])
            P_rot_var_pred.append(preds["e_P"])
        
        except:
            print(f"Cannnot get LightCurve of {target_star}, Try again")
            P_rot_pred.append(None)
            P_rot_var_pred.append(None)
            continue
        #download the first SPOC data


        
df_preds["P_rot_pred"] = P_rot_pred
df_preds["P_rot_var_pred"] = P_rot_var_pred     
        
df_preds
        

{'f': 1.377066883515752, 'P': 0.7261811404881999, 'amp': 0.0017466422705328139, 'ph': -0.16501125304087183, 'T0': 1411.0240476258605, 'offset': 0.9999764322275434, 'e_amp': 3.119895835092945e-05, 'e_ph': 0.002842865150259917, 'e_T0': 0.0020644350570699042, 'e_offset': 2.2060995015898884e-05, 'e_f': 0.0004960274106776764, 'e_P': 0.0002615746229258613}
{'f': 0.33485208366111685, 'P': 2.9863932428505904, 'amp': 0.006382653199946163, 'ph': 0.3059364144334744, 'T0': 1568.795495846915, 'offset': 1.000815346228361, 'e_amp': 4.008349809825173e-05, 'e_ph': 0.0009995039145797799, 'e_T0': 0.0029849117367037685, 'e_offset': 2.834331331895188e-05, 'e_f': 0.0001838933336071454, 'e_P': 0.0016400608975915391}
{'f': 0.29356918591853304, 'P': 3.4063520558915377, 'amp': 0.01146248092765418, 'ph': -0.4591106326084804, 'T0': 2175.796671969212, 'offset': 0.9994711099943504, 'e_amp': 5.023663701470698e-05, 'e_ph': 0.0006975286725157648, 'e_T0': 0.0023760282276673703, 'e_offset': 3.552266669710642e-05, 'e_f':

,target,P_rot,P_rot_var,P_rot_pred,P_rot_var_pred
0,TOI540,0.72610,0.00039,0.726181,0.000262
1,TOI837,3.004,0.053,2.986393,0.001640
2,TOI942,3.39,0.01,3.406352,0.001501
3,TOI251,3.84,0.48,3.837990,0.001528
4,TOI451,5.1,0.1,2.621496,0.001881
5,TOI1685,18.66,0.71,0.668251,0.002132
6,TOI776,34.4,2.0,10.000000,NaN
